In [ ]:
import pypsa
import pandas as pd
from matplotlib.pyplot import legend

### Read Network

In [ ]:
path_aldehyde_esc = 'aldehyde/resources/elec_s_20_ec_lc1.0_Co2L_730H_2030_0.069_esc.nc'

path_earth = 'pypsa-earth/networks/elec_s.nc'
path_earth_clustered = 'pypsa-earth/networks/elec_s_4.nc'

path_earth_sec = 'pypsa-earth-sec/results/test3/prenetworks/elec_s_4_ec_lc1_Co2L_144H_2030_0.071_DF_export.nc'
path_earth_sec = 'pypsa-earth-sec/results/test3/prenetworks/elec_s_4_ec_lc1_Co2L_144H_2030_0.071_DF_export.nc'

path_earth_sec_solved = 'pypsa-earth-sec/results/test3/postnetworks/elec_s_4_ec_lc1_Co2L_144H_2030_0.071_DF.nc'

#path_eur = 'pypsa-earth-sec/results/Runname_test/prenetworks/pypsa-eur/networks/elec_s_37_ec_lcopt_Co2L-3H.nc'
# path_africa = 'pypsa-africa/networks/elec_s_55_ec_lcopt_Co2L-3H.nc'
# path_africa_two = 'pypsa-africa/networks/elec_s.nc'


path_prefix = '../../../'

In [ ]:
n = pypsa.Network(path_prefix+path_earth_clustered)

### Investigate potentials

In [ ]:
for tech in ["onwind", "solar"]:
    potential = n.generators[n.generators.carrier == tech].p_nom_max
    print(f"The number of {tech} generators is {len(potential)}")
    print(f"The total {tech} potential is {potential.sum()*1e-3:.2f} GW")

In [ ]:
n.generators[n.generators.carrier=='solar']